In [73]:
!pip install wandb -qU

In [75]:
import wandb
wandb.login(key='61aac9b0b0cd9f6b611dda5a9137dd62870e5cdf')

2024-08-07 22:59:54.584976: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 22:59:54.611020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 22:59:55.089692: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] From /home/hl-turing/anaconda3/envs/cfgnn/lib/python3.11/site-packages/tensorflow/python/compat/v2_compat.py:98: disable_resource_variables (from tensorflow.python.ops.resource_variables_toggle) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term [deprecation.py _log_deprecation]


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 1, in <module>
    import wandb
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 38, in <module>
    wandb.login()
    ^^^^^^^^^^^
AttributeError: partially initialized module 'wandb' has no attribute 'login' (most likely due to a circular import)


ServiceStartProcessError: The wandb service process exited with 1. Ensure that `sys.executable` is a valid python interpreter. You can override it with the `_executable` setting or with the `WANDB__EXECUTABLE` environment variable.
{'command': ['/home/hl-turing/anaconda3/envs/cfgnn/bin/python', '-m', 'wandb', 'service', '--debug', '--port-filename', '/tmp/tmp03sz3k_8/port-250384.txt', '--pid', '250384', '--serve-sock'], 'sys_executable': '/home/hl-turing/anaconda3/envs/cfgnn/bin/python', 'which_python': '/home/hl-turing/anaconda3/envs/cfgnn/bin/python3', 'proc_out': '', 'proc_err': ''}

In [76]:
import wandb
wandb.login()

2024-08-07 23:00:07.120650: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 23:00:07.146320: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 23:00:07.619535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] From /home/hl-turing/anaconda3/envs/cfgnn/lib/python3.11/site-packages/tensorflow/python/compat/v2_compat.py:98: disable_resource_variables (from tensorflow.python.ops.resource_variables_toggle) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term [deprecation.py _log_deprecation]


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 1, in <module>
    import wandb
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 38, in <module>
    wandb.login()
    ^^^^^^^^^^^
AttributeError: partially initialized module 'wandb' has no attribute 'login' (most likely due to a circular import)


ServiceStartProcessError: The wandb service process exited with 1. Ensure that `sys.executable` is a valid python interpreter. You can override it with the `_executable` setting or with the `WANDB__EXECUTABLE` environment variable.
{'command': ['/home/hl-turing/anaconda3/envs/cfgnn/bin/python', '-m', 'wandb', 'service', '--debug', '--port-filename', '/tmp/tmpdw1tkrsa/port-250384.txt', '--pid', '250384', '--serve-sock'], 'sys_executable': '/home/hl-turing/anaconda3/envs/cfgnn/bin/python', 'which_python': '/home/hl-turing/anaconda3/envs/cfgnn/bin/python3', 'proc_out': '', 'proc_err': ''}

In [ ]:
import os
import random

import numpy as np
import torch
import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
from tqdm.auto import tqdm

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Track MetaData and HyperParameters

In [ ]:
config = dict(
    hidden_list_conv = [31,31,31],
    hidden_list_att = [50 for _ in range(7)], # for GIN and GAT
    alpha=0.2, # for GAT
    nheads=8, # for GAT
    dropout = 0.0,
    epochs=1000,
    classes=2,
    batch_size=1024,
    num_neig = [30]*3,
    learning_rate=0.002,
    clip = 2.0,
    dataset="AML",
    architecture="gin")

Now, let's define the overall pipeline,
which is pretty typical for model-training:

1. we first `make` a model, plus associated data and optimizer, then
2. we `train` the model accordingly and finally
3. `test` it to see how training went.

## Make Model

In [ ]:
from torch_geometric.loader import NeighborLoader
from carla.data.catalog import AMLtoGraph

import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.nn.dense import DenseGCNConv
from torch.nn.utils import clip_grad_norm_
from torcheval.metrics import MulticlassAccuracy, MulticlassF1Score, BinaryF1Score, BinaryAccuracy
from carla.models.catalog.parse_gnn import normalize_adj

### Models

In [ ]:
# Import Model
from carla.models.catalog.GAT_TORCH import GAT as gat_torch
from carla.models.catalog.GAT_COO_TORCH import GAT_COO as gat_coo_torch

from carla.models.catalog.GNN_TORCH import GCNSynthetic as gnn_torch
from carla.models.catalog.GNN_COO_TORCH import GNN_COO as gnn_coo_torch

from carla.models.catalog.GIN_TORCH import GIN as gin_torch
from carla.models.catalog.GIN_COO_TORCH import GIN_COO as gin_coo_torch

### Make

In [ ]:
def make(config):
    # Make the data
    datagraph = get_data(path = "/home/hl-turing/VSCodeProjects/Flavio/CARLA/data/AML_Laund_Clean.csv")
    train_loader = make_loader(datagraph, datagraph.train_mask, batch_size=config.batch_size, num_neig=config.num_neig)
    test_loader = make_loader(datagraph, datagraph.test_mask, batch_size=config.batch_size, num_neig=config.num_neig)

    # Make the model
    if config.architecture == 'gnn':
        model = gnn_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_conv=config.hidden_list_conv,
                nclass=config.classes,
                dropout=config.dropout,
        ).to(device)
    elif config.architecture == "gat":
        #hidden_list_att = [50 for _ in range(3)]
        model = gat_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_att=config.hidden_list_att,  # da parametrizzare
                hid_list_conv=config.hidden_list_conv,
                nclass=config.classes,
                dropout=config.dropout,
                alpha=config.alpha,
                nheads=config.nheads,
            ).to(device)
    elif config.architecture == "gin":
            # forse definire hidden_list_att DA PARAMETRIZZARE
        #hidden_list_att = [50 for _ in range(7)]
        model = gin_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_gin=config.hidden_list_att,  # da parametrizzare
                hid_list_conv=config.hidden_list_conv,
                nclass=config.classes,
                dropout=config.dropout,
                alpha=config.alpha,
                nheads=config.nheads
            ).to(device)
    elif config.architecture == 'gnn_coo':
        model = gnn_coo_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_conv=config.hidden_list_conv,
                nclass=config.classes,
                dropout=config.dropout,
        ).to(device)
        
    elif config.architecture == "gat_coo":
            # forse definire hidden_list_att DA PARAMETRIZZARE
        #hidden_list_att = [31 for _ in range(4)]
        model = gat_coo_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_att=config.hidden_list_att,  # da parametrizzare
                hid_list_conv=config.hidden_list_conv,
                nclass=config.classes,
                dropout=config.dropout,
                alpha=config.alpha,
                nheads=len(config.hidden_list_att),
            ).to(device)
    elif config.architecture == "gin_coo":
            # forse definire hidden_list_att DA PARAMETRIZZARE
        #hidden_list_att = [31 for _ in range(4)]
        model = gin_coo_torch(
                nfeat=len(datagraph.x[0]),
                hid_list_gin=config.hidden_list_att,  # da parametrizzare
                hid_list_conv=config.hidden_list_conv,
                edge_dim = datagraph.edge_attr.shape[1],
                nclass=config.classes,
                dropout=config.dropout
            ).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    return model, train_loader, test_loader, criterion, optimizer

def get_data(path):
    data = AMLtoGraph(path)
    datagraph = data.construct_GraphData()
    return datagraph

def make_loader(datagraph, mask, batch_size, num_neig):
    loader = NeighborLoader(
        datagraph,
        num_neighbors=num_neig,
        batch_size=batch_size,
        input_nodes=mask
    )
    return loader

## Train and Test

### Train

In [ ]:
def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)
    
    metric = BinaryAccuracy()
    # f1score = MulticlassF1Score(num_classes=2, average="macro")
    f1score = BinaryF1Score()
    t_total = time.time()
    # Run training and track with wandb

    for epoch in tqdm(range(config.epochs)):
        t = time.time()
        average_epoch_loss = 0.0
        model.train()
        for data in loader:
            data.to(device)

            adj = _create_adj_mat(data)
            norm_adj = normalize_adj(adj).to(device)
            features = torch.tensor(data.x).squeeze().to(device)
            labels = torch.tensor(data.y).squeeze().long().to(device)

            node_idx = [i for i in range(0, len(data.y))]
            idx_train = torch.masked_select(torch.Tensor(node_idx, device="cpu"), data.train_mask.cpu())
            # idx_test = torch.masked_select(torch.Tensor(node_idx, device="cpu"), data.test_mask.cpu())
            idx_train = idx_train.type(torch.int64)
            optimizer.zero_grad()
            if config.architecture == "gnn" or config.architecture == "gat" or config.architecture == "gin":
                features, norm_adj = features.to(device), norm_adj.to(device)
                output = model(features, norm_adj).squeeze()
            ## coo list 
            elif config.architecture == "gnn_coo" or config.architecture == "gat_coo" or config.architecture == "gin_coo":
                features =  features.to(device)
                edge_index, edge_attr = data.edge_index.to(device), data.edge_attr.to(device)
                output = model(features, edge_index, edge_attr)
            elif config.architecture == "gUnet_coo":
                features =  features.to(device)
                edge_index, edge_attr = data.edge_index.to(device), data.edge_attr.to(device)
                output = model(features, edge_index)

            loss_train = F.cross_entropy(output[idx_train], labels[idx_train])            
            y_pred = torch.argmax(output, dim=1)

            accuracy = metric.update(y_pred[idx_train], labels[idx_train])
            f1 = f1score.update(y_pred[idx_train], labels[idx_train])
            average_epoch_loss += loss_train.item()
            
            loss_train.backward()
            
            clip_grad_norm_(model.parameters(), config.clip)
            optimizer.step()
        
        avg_loss = average_epoch_loss/len(loader)
        # Report metrics 
        train_log(avg_loss, epoch, accuracy, f1, t)
        
        
        
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    





def train_log(avg_loss, epoch, accuracy, f1, t):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": avg_loss, "accuracy":accuracy.compute(), "F1": f1.compute() })
    print(
            "Epoch: {:04d}".format(epoch + 1),
            "loss_train: {:.4f}".format(avg_loss),
            "acc_train: {:.4f}".format(accuracy.compute()),
            "f1_train: {:.4f}".format(f1.compute()),
            "time: {:.4f}s".format(time.time() - t),
        )
    
    
    
def _create_adj_mat(data_graph):
    edges_index = data_graph.edge_index
    row_indices = edges_index[0]
    col_indices = edges_index[1]
    values = torch.ones(len(edges_index[0]))  # valori tutti a uno
    size = torch.Size([len(data_graph.x), len(data_graph.x)])
    sparse_matrix = torch.sparse_coo_tensor(
        torch.stack([row_indices, col_indices]), values, size=size, device="cuda"
    )
    adj_matrix = sparse_matrix.to_dense()
    return adj_matrix

In [ ]:
def model_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project="pytorch-gnn", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)

      # and test its final performance
      # test(model, test_loader)
      
    # Save the model checkpoint

      # torch.onnx.export(model, "model.onnx")
      #wandb.save("model.onnx")
    return model

In [ ]:
model = model_pipeline(config)

/home/hl-turing/VSCodeProjects/Flavio/CARLA/carla/data/catalog/graph_catalog.py:294: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self._data_table)


GIN(
  (layers_conv): ModuleList(
    (0): DenseGINConv(nn=Sequential(
      (0): Linear(in_features=31, out_features=50, bias=True)
      (1): ReLU()
      (2): Linear(in_features=50, out_features=50, bias=True)
    ))
    (1-6): 6 x DenseGINConv(nn=Sequential(
      (0): Linear(in_features=50, out_features=50, bias=True)
      (1): ReLU()
      (2): Linear(in_features=50, out_features=50, bias=True)
    ))
    (7): DenseGCNConv(50, 31)
    (8-9): 2 x DenseGCNConv(31, 31)
    (10): Linear(in_features=31, out_features=2, bias=True)
  )
)


  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_233384/2451238061.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(data.x).squeeze().to(device)
/tmp/ipykernel_233384/2451238061.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(data.y).squeeze().long().to(device)
  0%|          | 1/1000 [00:00<08:43,  1.91it/s]

Epoch: 0001 loss_train: 0.7082 acc_train: 0.5792 f1_train: 0.3838 time: 0.5240s


  0%|          | 2/1000 [00:01<08:45,  1.90it/s]

...

  4%|▎         | 37/1000 [00:18<07:48,  2.06it/s]

Epoch: 0037 loss_train: 0.2713 acc_train: 0.7739 f1_train: 0.2369 time: 0.4791s


  4%|▎         | 37/1000 [00:18<07:59,  2.01it/s]
Traceback (most recent call last):
  File "/tmp/ipykernel_233384/1643529747.py", line 13, in model_pipeline
    train(model, train_loader, criterion, optimizer, config)
  File "/tmp/ipykernel_233384/2451238061.py", line 19, in train
    norm_adj = normalize_adj(adj).to(device)
               ^^^^^^^^^^^^^^^^^^
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/carla/models/catalog/parse_gnn.py", line 143, in normalize_adj
    D_tilde = torch.pow(get_degree_matrix(A_tilde), -0.5)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/carla/models/catalog/parse_gnn.py", line 133, in get_degree_matrix
    return torch.diag(sum(adj))
                      ^^^^^^^^
KeyboardInterrupt


F1,█▇▆▅▄▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▃▃▃▃▂▂▂▂▂▃▂▂▂▅▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▁▂▁▁
F1,0.23694
accuracy,0.77389
epoch,36
loss,0.27131


KeyboardInterrupt: 

In [ ]:
len([[31]*i for i in range(3)] + [[50]*i for i in range(3)] + [[100]*i for i in range(3)])

9

# HyperParameterTuning

In [ ]:

sweep_config = {
    'method': 'bayes',
    
    'metric':{
        'goal': 'minimize',
        'name': 'loss'
    },
    
    'parameters':{
        'hidden_list_conv': {
            'values': [[31]*i for i in range(7)] + [[50]*i for i in range(7)] + [[100]*i for i in range(7)]
        },
        
        'hidden_list_att':{
            'values': [[31]*i for i in range(7)] + [[50]*i for i in range(7)] + [[100]*i for i in range(7)]
        },
        
        'alpha':{
            'max': 0.4,
            'min': 0.1,
            'distribution': 'uniform'
        },
        
        'nheads':{
            'max': 8,
            'min': 1,
            'distribution': 'int_uniform'   
        },
        
        'dropout':{
            'values': [0.0]
        },
        
        
        'epochs':{
            'max': 2000,
            'min': 500,
            'distribution': 'int_uniform'   
        },
        
        'classes':{
            'values': [2]
        },
        
        'batch_size':{
            'max': 2048,
            'min': 512,
            'distribution': 'int_uniform'   
        },
        
        'num_neig':{
            'values': [[31]*i for i in range(4)] + [[50]*i for i in range(4)] + [[100]*i for i in range(4)]
        },
        
        'learning_rate':{
            'values': [0.01, 0.02, 0.001, 0.002, 0.0001, 0.0002]
        },

        'clip':{
            'max': 4.0,
            'min': 1.0,
            'distribution': 'int_uniform'   
        },
        
        'dataset':{
            'values': ['AML']
        },

        'architecture':{
            'values': ['gnn'],
            'distribution': 'categorical'
        }
        
    }
    

}

In [ ]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'alpha': {'distribution': 'uniform', 'max': 0.4, 'min': 0.1},
                'architecture': {'distribution': 'categorical',
                                 'values': ['gnn']},
                'batch_size': {'distribution': 'int_uniform',
                               'max': 2048,
                               'min': 512},
                'classes': {'values': [2]},
                'clip': {'distribution': 'int_uniform', 'max': 4.0, 'min': 1.0},
                'dataset': {'values': ['AML']},
                'dropout': {'values': [0.0]},
                'epochs': {'distribution': 'int_uniform',
                           'max': 2000,
                           'min': 500},
                'hidden_list_att': {'values': [[],
                                               [31],
                                               [31, 31],
                                               [31, 31, 31],
       

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-gnn")

Create sweep with ID: xuzk9tp7
Sweep URL: https://wandb.ai/leonardvincentramil-sapienza-universit-di-roma/pytorch-gnn/sweeps/xuzk9tp7


In [ ]:
def train_sweep(hyperparameters=None):

    # tell wandb to get started
    with wandb.init(config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)

      # and test its final performance
      # test(model, test_loader)
      
    # Save the model checkpoint

      # torch.onnx.export(model, "model.onnx")
      #wandb.save("model.onnx")

In [ ]:
wandb.agent(sweep_id, train_sweep, count=5)

wandb: Agent Starting Run: 1neomm17 with config:
wandb: 	alpha: 0.36983818162277937
wandb: 	architecture: gnn
wandb: 	batch_size: 1999
wandb: 	classes: 2
wandb: 	clip: 3
wandb: 	dataset: AML
wandb: 	dropout: 0
wandb: 	epochs: 1775
wandb: 	hidden_list_att: [50, 50, 50, 50, 50]
wandb: 	hidden_list_conv: [50]
wandb: 	learning_rate: 0.0002
wandb: 	nheads: 8
wandb: 	num_neig: [100]
2024-08-07 22:56:29.224233: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 22:56:29.249942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriat

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] From /home/hl-turing/anaconda3/envs/cfgnn/lib/python3.11/site-packages/tensorflow/python/compat/v2_compat.py:98: disable_resource_variables (from tensorflow.python.ops.resource_variables_toggle) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term [deprecation.py _log_deprecation]


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 1, in <module>
    import wandb
  File "/home/hl-turing/VSCodeProjects/Flavio/CARLA/wandb.py", line 38, in <module>
    wandb.login()
    ^^^^^^^^^^^
AttributeError: partially initialized module 'wandb' has no attribute 'login' (most likely due to a circular import)
wandb: ERROR Run 1neomm17 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/hl-turing/anaconda3/envs/cfgnn/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_250384/1996428651.py", line 4, in train_sweep
wandb: ERROR     with wandb.init(config=hyperparameters):
wandb: ERROR          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/home/hl-turing/anaconda3/envs/cfgnn/lib/python3.11/sit